In [1]:
import time
import argparse
import numpy as np

import torch
import torch.optim as optim

import torch.nn as nn

from utils import load_data,normalize,toy_data,nmi_score,modularity_matrix,modularity
from models import GNN

import community as community_louvain
from networkx import from_numpy_matrix
import networkx as nx

from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module

from sklearn.cluster import SpectralClustering

import pycombo

import pandas as pd

torch.set_printoptions(sci_mode=False)

from utils import doublerelu

import networkx as nx

In [2]:
import warnings
warnings.filterwarnings("ignore") 

In [3]:
cuda = torch.cuda.is_available()
weight_decay = 10e-2
epochs = 1000
seed = 17

In [4]:
class GNN1Layer(Module):

    def __init__(self, batch_size, in_features, out_features):
        super(GNN1Layer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.batch_size = batch_size

        weight1_eye = torch.FloatTensor(torch.eye(in_features, out_features))
        weight1_eye = weight1_eye.reshape((1, in_features, out_features))
        weight1_eye = weight1_eye.repeat(batch_size, 1, 1)
        self.weight1 = Parameter(weight1_eye)
        self.weight2 = Parameter(torch.zeros(batch_size, in_features, out_features))

    def forward(self, input, adj):
        v1 = torch.bmm(input, self.weight1)
        v2 = torch.bmm(torch.bmm(adj, input), self.weight2)
        output = v1 + v2
        return output

In [5]:
class GNN2Layer(Module):

    def __init__(self, batch_size, in_features, out_features):
        super(GNN2Layer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.batch_size = batch_size
        weight1_eye = torch.FloatTensor(torch.eye(in_features, in_features))
        weight1_eye = weight1_eye.reshape((1, in_features, in_features))
        weight1_eye = weight1_eye.repeat(batch_size, 1, 1)
        weight1_rand = torch.empty(batch_size,in_features,out_features-in_features)
        torch.nn.init.xavier_uniform_(weight1_rand, gain=1.0)
        self.weight1 = Parameter(torch.cat((weight1_eye,weight1_rand),dim=2))
        self.weight2 = Parameter(torch.zeros(batch_size, in_features, out_features))

    def forward(self, input, adj):
        v1 = torch.bmm(input, self.weight1)
        v2 = torch.bmm(torch.bmm(adj, input), self.weight2)
        output = v1 + v2
        return output

In [6]:
class GNN1(nn.Module):

    def __init__(self, batch_size, nfeat, ndim, hidden):
        super(GNN1, self).__init__()

        self.gc1 = GNN1Layer(batch_size, ndim, ndim)

    def forward(self, x, adj):
        x = doublerelu(self.gc1(x, adj))
        x = x/x.sum(axis=2).unsqueeze(2) #normalize st sum = 1
        return x

In [7]:
class GNN2(nn.Module):

    def __init__(self, batch_size, nfeat, ndim, hidden):
        super(GNN2, self).__init__()

        self.gc1 = GNN2Layer(batch_size, ndim, hidden)
        self.gc2 = GNN1Layer(batch_size, hidden, ndim)

    def forward(self, x, adj):
        x = doublerelu(self.gc1(x, adj))
        x = doublerelu(self.gc2(x, adj))
        x = x/x.sum(axis=2).unsqueeze(2) #normalize st sum = 1
        return x

In [8]:
def community_detection(G,method,GNN,features=None,hidden=1,seed=seed):
    
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed(seed)
    
    adj = nx.to_numpy_matrix(G)
    adj = np.expand_dims(adj, axis=0)
        
    #Combo
    combo_partition = pycombo.execute(G)
    nb_comm = max(combo_partition[0].values())+1
    combo_mod = combo_partition[1]

    adj_norm = normalize(adj)

    adj = torch.FloatTensor(np.array(adj))
    adj_norm = torch.FloatTensor(np.array(adj_norm))
    
        
    if type(features) == type(None):

        # features
        if method == "louvain":
            partition = community_louvain.best_partition(G)
            #get binary matrix of the partition
            nb_community = max(list(partition.values())) + 1
            communities =  np.array(list(partition.values())).reshape(-1)
            C = np.eye(nb_community)[communities]
            C = torch.FloatTensor(C)
            #perturbations
            C = perturb(C)
        elif method == "spectral":
            partition = spectralPartition(adj,nb_comm)
            C = partition.float()
            nb_community = C.shape[1]

        features = torch.FloatTensor(C)
        features = features.unsqueeze(0)
    
    if method == "louvain":
        lr = 0.002
    elif method == "spectral":
        lr = 0.002
        
    nb_community = features.shape[-1]
    
    Q = modularity_matrix(adj)
    
    # Model and optimizer

    model = GNN(batch_size=adj.shape[0],
                nfeat=adj.shape[1],
                ndim=nb_community,
                hidden=hidden)

    if cuda:
        model.cuda()
        features = features.cuda()
        adj = adj.cuda()
        adj_norm = adj_norm.cuda()
        Q = Q.cuda()

    # Train model
    t_total = time.time()

    optimizer = optim.Adam(model.parameters(),
                           lr=lr, weight_decay=weight_decay)
    

    for epoch in range(epochs):

        t = time.time()
        model.train()
        optimizer.zero_grad()

        C = model(features, adj_norm)
        loss = modularity(C,Q)

        loss.backward(retain_graph=True)

        optimizer.step()

        if epoch == 0:
            best_loss = loss
            init_mod = loss
            best_C = C
        else:
            if loss < best_loss:
                best_loss = loss
                best_C = C

        if epoch % 100 == 0:
            print('Epoch: {:04d}'.format(epoch + 1),
                  'Modularity: {:.8f}'.format(-best_loss.item()),
                  'time: {:.4f}s'.format(time.time() - t))

    print("Optimization Finished!")
    print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
    
    return nb_community,-init_mod,-best_loss,combo_mod, best_C

In [9]:
def perturb(A):
    p = 0.4
    epsilon = torch.rand(A.shape).uniform_(0, p) - torch.rand(A.shape).uniform_(0, p)
    return torch.clip(A + epsilon,0,1)

In [10]:
def spectralPartition(A,nb_comm):
    A = A[0] # TODO 3D
    clustering = SpectralClustering(n_clusters=nb_comm,assign_labels='discretize',random_state=0).fit(A)
    clusters = list(clustering.labels_)
    partitions = torch.tensor(clusters)
    return torch.nn.functional.one_hot(partitions)

In [11]:
def graphs():
    G = [nx.read_pajek('data/davidcopperfield.net'),nx.read_pajek('data/jazz.net'),nx.read_pajek('data/dolphins.net')]
    return G

In [12]:
def exp1(hidden):
    G = graphs()
    methods = ["louvain","spectral"]
    finetuned_mods = []
    networks = ["David Copperfield","David Copperfield","Jazz Musicians","Jazz Musicians","Dolphins Social Network","Dolphins Social Network"]
    for network in G:
        for method in methods:
            nb_community,init_mod,finetuned_mod,combo_mod,features = community_detection(network,method,GNN1,hidden=hidden)
            finetuned_mods.append(finetuned_mod.cpu().detach().numpy()[0][0])
            
    return finetuned_mods

In [13]:
def exp2(hidden):
    G = graphs()
    methods = ["louvain","spectral"]
    nb_communities = []
    init_mods = []
    finetuned_mods = []
    combo_mods = []
    methodss = []
    networks = ["David Copperfield","David Copperfield","Jazz Musicians","Jazz Musicians","Dolphins Social Network","Dolphins Social Network"]
    hidden_dim = []
    one_layer_mod = exp1(hidden)
    for network in G:
        for method in methods:
            flag = True
            for i in range(1):
                seed = np.random.randint(10000)
                nb_community,init_mod,finetuned_mod,combo_mod,features = community_detection(network,method,GNN1,hidden=hidden,seed=seed)
                initial_mod = torch.clone(init_mod)
                nb_community,init_mod,finetuned_mod,combo_mod,features = community_detection(network,method,GNN2,features=features.detach(),hidden=nb_community+hidden,seed=seed)
                while init_mod<finetuned_mod:
                    nb_community,init_mod,finetuned_mod,combo_mod,features = community_detection(network,method,GNN2,features=features.detach(),hidden=nb_community+hidden,seed=seed)
                if flag:
                    nb_communities.append(nb_community)
                    init_mods.append(initial_mod.cpu().detach().numpy()[0][0])
                    finetuned_mods.append(finetuned_mod.cpu().detach().numpy()[0][0])
                    combo_mods.append(combo_mod)
                    methodss.append(method)
                    hidden_dim.append(nb_community+hidden)
                    flag = False
                else:
                    if finetuned_mod.cpu().detach().numpy()[0][0] > finetuned_mods[-1]:
                        finetuned_mods[-1] = finetuned_mod.cpu().detach().numpy()[0][0]
            
            
    df = pd.DataFrame([networks,methodss,nb_communities,init_mods,one_layer_mod,combo_mods,hidden_dim,finetuned_mods]).transpose()
    df.columns = ["Network","Method","Communities","Initial Modularity","1 Layer Finetuned Moduclarity","Combo Modularity","Hidden Dimensionality","2 Layer Finetuned Moduclarity"] 
    return df 

In [14]:
dfs = exp2(1)
for hidden in range(2,11):
    dfs = pd.concat([dfs,exp2(hidden)[['Hidden Dimensionality','2 Layer Finetuned Moduclarity']]],axis=1)

Epoch: 0001 Modularity: 0.11401027 time: 0.3769s
Epoch: 0101 Modularity: 0.27859122 time: 0.0156s
Epoch: 0201 Modularity: 0.28211001 time: 0.0000s
Epoch: 0301 Modularity: 0.28426713 time: 0.0156s
Epoch: 0401 Modularity: 0.28530049 time: 0.0000s
Epoch: 0501 Modularity: 0.28600383 time: 0.0000s
Epoch: 0601 Modularity: 0.28602374 time: 0.0000s
Epoch: 0701 Modularity: 0.28602397 time: 0.0000s
Epoch: 0801 Modularity: 0.28602469 time: 0.0130s
Epoch: 0901 Modularity: 0.28602469 time: 0.0000s
Optimization Finished!
Total time elapsed: 2.4197s
Epoch: 0001 Modularity: 0.03314324 time: 0.0000s
Epoch: 0101 Modularity: 0.06331336 time: 0.0000s
Epoch: 0201 Modularity: 0.08956972 time: 0.0000s
Epoch: 0301 Modularity: 0.11138584 time: 0.0000s
Epoch: 0401 Modularity: 0.11138584 time: 0.0000s
Epoch: 0501 Modularity: 0.11138584 time: 0.0000s
Epoch: 0601 Modularity: 0.11138584 time: 0.0000s
Epoch: 0701 Modularity: 0.11138584 time: 0.0000s
Epoch: 0801 Modularity: 0.11138584 time: 0.0000s
Epoch: 0901 Modula

Epoch: 0201 Modularity: 0.22632997 time: 0.0000s
Epoch: 0301 Modularity: 0.22639748 time: 0.0160s
Epoch: 0401 Modularity: 0.22655201 time: 0.0156s
Epoch: 0501 Modularity: 0.22691286 time: 0.0000s
Epoch: 0601 Modularity: 0.22842383 time: 0.0000s
Epoch: 0701 Modularity: 0.22842383 time: 0.0156s
Epoch: 0801 Modularity: 0.22842383 time: 0.0157s
Epoch: 0901 Modularity: 0.22842383 time: 0.0000s
Optimization Finished!
Total time elapsed: 3.1098s
Epoch: 0001 Modularity: 0.22842383 time: 0.0156s
Epoch: 0101 Modularity: 0.22855748 time: 0.0000s
Epoch: 0201 Modularity: 0.22865961 time: 0.0000s
Epoch: 0301 Modularity: 0.22890130 time: 0.0000s
Epoch: 0401 Modularity: 0.22983786 time: 0.0000s
Epoch: 0501 Modularity: 0.23260427 time: 0.0000s
Epoch: 0601 Modularity: 0.23265374 time: 0.0000s
Epoch: 0701 Modularity: 0.23268792 time: 0.0000s
Epoch: 0801 Modularity: 0.23268792 time: 0.0000s
Epoch: 0901 Modularity: 0.23268792 time: 0.0150s
Optimization Finished!
Total time elapsed: 2.8554s
Epoch: 0001 Modu

Epoch: 0401 Modularity: 0.43551508 time: 0.0000s
Epoch: 0501 Modularity: 0.43551508 time: 0.0000s
Epoch: 0601 Modularity: 0.43551511 time: 0.0000s
Epoch: 0701 Modularity: 0.43551511 time: 0.0000s
Epoch: 0801 Modularity: 0.43551511 time: 0.0000s
Epoch: 0901 Modularity: 0.43551511 time: 0.0000s
Optimization Finished!
Total time elapsed: 2.5736s
Epoch: 0001 Modularity: 0.43551511 time: 0.0000s
Epoch: 0101 Modularity: 0.43551511 time: 0.0157s
Epoch: 0201 Modularity: 0.43551511 time: 0.0000s
Epoch: 0301 Modularity: 0.43551511 time: 0.0000s
Epoch: 0401 Modularity: 0.43551511 time: 0.0000s
Epoch: 0501 Modularity: 0.43551511 time: 0.0000s
Epoch: 0601 Modularity: 0.43551511 time: 0.0156s
Epoch: 0701 Modularity: 0.43551511 time: 0.0000s
Epoch: 0801 Modularity: 0.43551511 time: 0.0000s
Epoch: 0901 Modularity: 0.43551511 time: 0.0000s
Optimization Finished!
Total time elapsed: 2.4646s
Epoch: 0001 Modularity: 0.06414337 time: 0.0000s
Epoch: 0101 Modularity: 0.07505678 time: 0.0000s
Epoch: 0201 Modu

Epoch: 0601 Modularity: 0.47808629 time: 0.0156s
Epoch: 0701 Modularity: 0.47808629 time: 0.0000s
Epoch: 0801 Modularity: 0.47808629 time: 0.0000s
Epoch: 0901 Modularity: 0.47808629 time: 0.0156s
Optimization Finished!
Total time elapsed: 2.5355s
Epoch: 0001 Modularity: 0.47808629 time: 0.0000s
Epoch: 0101 Modularity: 0.47808629 time: 0.0000s
Epoch: 0201 Modularity: 0.47808629 time: 0.0156s
Epoch: 0301 Modularity: 0.47808629 time: 0.0000s
Epoch: 0401 Modularity: 0.47808629 time: 0.0000s
Epoch: 0501 Modularity: 0.47808629 time: 0.0000s
Epoch: 0601 Modularity: 0.47808629 time: 0.0000s
Epoch: 0701 Modularity: 0.47808629 time: 0.0095s
Epoch: 0801 Modularity: 0.47808629 time: 0.0000s
Epoch: 0901 Modularity: 0.47808629 time: 0.0000s
Optimization Finished!
Total time elapsed: 2.3391s
Epoch: 0001 Modularity: 0.11401027 time: 0.0000s
Epoch: 0101 Modularity: 0.27859122 time: 0.0000s
Epoch: 0201 Modularity: 0.28211001 time: 0.0000s
Epoch: 0301 Modularity: 0.28426713 time: 0.0000s
Epoch: 0401 Modu

Epoch: 0801 Modularity: 0.29602253 time: 0.0000s
Epoch: 0901 Modularity: 0.29602253 time: 0.0000s
Optimization Finished!
Total time elapsed: 2.3840s
Epoch: 0001 Modularity: 0.29602253 time: 0.0156s
Epoch: 0101 Modularity: 0.29602253 time: 0.0000s
Epoch: 0201 Modularity: 0.29602253 time: 0.0000s
Epoch: 0301 Modularity: 0.29602253 time: 0.0000s
Epoch: 0401 Modularity: 0.29602253 time: 0.0000s
Epoch: 0501 Modularity: 0.29602253 time: 0.0000s
Epoch: 0601 Modularity: 0.29602253 time: 0.0000s
Epoch: 0701 Modularity: 0.29602253 time: 0.0000s
Epoch: 0801 Modularity: 0.29602253 time: 0.0000s
Epoch: 0901 Modularity: 0.29602253 time: 0.0156s
Optimization Finished!
Total time elapsed: 2.4166s
Epoch: 0001 Modularity: 0.01335085 time: 0.0000s
Epoch: 0101 Modularity: 0.05648800 time: 0.0000s
Epoch: 0201 Modularity: 0.09383394 time: 0.0000s
Epoch: 0301 Modularity: 0.09794273 time: 0.0000s
Epoch: 0401 Modularity: 0.10719595 time: 0.0000s
Epoch: 0501 Modularity: 0.10719595 time: 0.0000s
Epoch: 0601 Modu

...

Epoch: 0101 Modularity: 0.22600460 time: 0.0000s
Epoch: 0201 Modularity: 0.22600582 time: 0.0000s
Epoch: 0301 Modularity: 0.22600612 time: 0.0000s
Epoch: 0401 Modularity: 0.22600728 time: 0.0000s
Epoch: 0501 Modularity: 0.22601047 time: 0.0000s
Epoch: 0601 Modularity: 0.22601844 time: 0.0000s
Epoch: 0701 Modularity: 0.22604337 time: 0.0000s
Epoch: 0801 Modularity: 0.22613806 time: 0.0000s
Epoch: 0901 Modularity: 0.22613806 time: 0.0000s
Optimization Finished!
Total time elapsed: 2.0401s
Epoch: 0001 Modularity: 0.22613806 time: 0.0000s
Epoch: 0101 Modularity: 0.22614688 time: 0.0000s
Epoch: 0201 Modularity: 0.22614688 time: 0.0000s
Epoch: 0301 Modularity: 0.22614688 time: 0.0156s
Epoch: 0401 Modularity: 0.22614843 time: 0.0156s
Epoch: 0501 Modularity: 0.22616065 time: 0.0000s
Epoch: 0601 Modularity: 0.22618522 time: 0.0000s
Epoch: 0701 Modularity: 0.22627819 time: 0.0000s
Epoch: 0801 Modularity: 0.22627819 time: 0.0000s
Epoch: 0901 Modularity: 0.22627819 time: 0.0000s
Optimization Finis

Epoch: 0301 Modularity: 0.30041957 time: 0.0000s
Epoch: 0401 Modularity: 0.30041957 time: 0.0000s
Epoch: 0501 Modularity: 0.30041957 time: 0.0000s
Epoch: 0601 Modularity: 0.30041957 time: 0.0000s
Epoch: 0701 Modularity: 0.30041957 time: 0.0000s
Epoch: 0801 Modularity: 0.30041957 time: 0.0000s
Epoch: 0901 Modularity: 0.30041957 time: 0.0000s
Optimization Finished!
Total time elapsed: 2.5758s
Epoch: 0001 Modularity: 0.26544726 time: 0.0000s
Epoch: 0101 Modularity: 0.50538939 time: 0.0000s
Epoch: 0201 Modularity: 0.50794470 time: 0.0155s
Epoch: 0301 Modularity: 0.50906670 time: 0.0000s
Epoch: 0401 Modularity: 0.50935483 time: 0.0000s
Epoch: 0501 Modularity: 0.50935483 time: 0.0000s
Epoch: 0601 Modularity: 0.50935483 time: 0.0000s
Epoch: 0701 Modularity: 0.50935483 time: 0.0000s
Epoch: 0801 Modularity: 0.50935483 time: 0.0000s
Epoch: 0901 Modularity: 0.50935483 time: 0.0000s
Optimization Finished!
Total time elapsed: 1.8195s
Epoch: 0001 Modularity: 0.50935483 time: 0.0000s
Epoch: 0101 Modu

Epoch: 0501 Modularity: 0.29389271 time: 0.0000s
Epoch: 0601 Modularity: 0.29393941 time: 0.0000s
Epoch: 0701 Modularity: 0.29416192 time: 0.0000s
Epoch: 0801 Modularity: 0.29416192 time: 0.0000s
Epoch: 0901 Modularity: 0.29416192 time: 0.0156s
Optimization Finished!
Total time elapsed: 2.1177s
Epoch: 0001 Modularity: 0.29416192 time: 0.0000s
Epoch: 0101 Modularity: 0.29416192 time: 0.0000s
Epoch: 0201 Modularity: 0.29416192 time: 0.0156s
Epoch: 0301 Modularity: 0.29416192 time: 0.0000s
Epoch: 0401 Modularity: 0.29416192 time: 0.0166s
Epoch: 0501 Modularity: 0.29416192 time: 0.0000s
Epoch: 0601 Modularity: 0.29416192 time: 0.0000s
Epoch: 0701 Modularity: 0.29416192 time: 0.0156s
Epoch: 0801 Modularity: 0.29416192 time: 0.0000s
Epoch: 0901 Modularity: 0.29416192 time: 0.0000s
Optimization Finished!
Total time elapsed: 2.0233s
Epoch: 0001 Modularity: 0.03314324 time: 0.0000s
Epoch: 0101 Modularity: 0.06331336 time: 0.0000s
Epoch: 0201 Modularity: 0.08956972 time: 0.0030s
Epoch: 0301 Modu

Epoch: 0801 Modularity: 0.11545191 time: 0.0000s
Epoch: 0901 Modularity: 0.11545191 time: 0.0000s
Optimization Finished!
Total time elapsed: 1.2227s
Epoch: 0001 Modularity: 0.11545191 time: 0.0000s
Epoch: 0101 Modularity: 0.23092982 time: 0.0156s
Epoch: 0201 Modularity: 0.23221511 time: 0.0000s
Epoch: 0301 Modularity: 0.23221511 time: 0.0000s
Epoch: 0401 Modularity: 0.23221511 time: 0.0000s
Epoch: 0501 Modularity: 0.23221511 time: 0.0000s
Epoch: 0601 Modularity: 0.23221511 time: 0.0156s
Epoch: 0701 Modularity: 0.23221511 time: 0.0000s
Epoch: 0801 Modularity: 0.23221511 time: 0.0035s
Epoch: 0901 Modularity: 0.23221511 time: 0.0000s
Optimization Finished!
Total time elapsed: 1.8522s
Epoch: 0001 Modularity: 0.23221511 time: 0.0000s
Epoch: 0101 Modularity: 0.23789641 time: 0.0000s
Epoch: 0201 Modularity: 0.24219057 time: 0.0000s
Epoch: 0301 Modularity: 0.24453399 time: 0.0000s
Epoch: 0401 Modularity: 0.24837276 time: 0.0000s
Epoch: 0501 Modularity: 0.24915211 time: 0.0000s
Epoch: 0601 Modu

Epoch: 0001 Modularity: 0.51931584 time: 0.0000s
Epoch: 0101 Modularity: 0.51931584 time: 0.0000s
Epoch: 0201 Modularity: 0.51936936 time: 0.0000s
Epoch: 0301 Modularity: 0.51939458 time: 0.0000s
Epoch: 0401 Modularity: 0.51945430 time: 0.0000s
Epoch: 0501 Modularity: 0.51956117 time: 0.0000s
Epoch: 0601 Modularity: 0.51980543 time: 0.0000s
Epoch: 0701 Modularity: 0.52083039 time: 0.0000s
Epoch: 0801 Modularity: 0.52208507 time: 0.0000s
Epoch: 0901 Modularity: 0.52208507 time: 0.0000s
Optimization Finished!
Total time elapsed: 1.9294s
Epoch: 0001 Modularity: 0.52208507 time: 0.0000s
Epoch: 0101 Modularity: 0.52222896 time: 0.0000s
Epoch: 0201 Modularity: 0.52235401 time: 0.0000s
Epoch: 0301 Modularity: 0.52268559 time: 0.0000s
Epoch: 0401 Modularity: 0.52333766 time: 0.0000s
Epoch: 0501 Modularity: 0.52333766 time: 0.0000s
Epoch: 0601 Modularity: 0.52333766 time: 0.0000s
Epoch: 0701 Modularity: 0.52333766 time: 0.0000s
Epoch: 0801 Modularity: 0.52333766 time: 0.0000s
Epoch: 0901 Modula

In [15]:
dfs

,Network,Method,Communities,Initial Modularity,1 Layer Finetuned Moduclarity,Combo Modularity,Hidden Dimensionality,2 Layer Finetuned Moduclarity,Hidden Dimensionality,2 Layer Finetuned Moduclarity,...,Hidden Dimensionality,2 Layer Finetuned Moduclarity,Hidden Dimensionality,2 Layer Finetuned Moduclarity,Hidden Dimensionality,2 Layer Finetuned Moduclarity,Hidden Dimensionality,2 Layer Finetuned Moduclarity,Hidden Dimensionality,2 Layer Finetuned Moduclarity
0,David Copperfield,louvain,7,0.111281,0.286025,0.306455,8,0.294162,9,0.296023,...,13,0.296819,14,0.296703,15,0.296022,16,0.296022,17,0.294162
1,David Copperfield,spectral,6,0.033143,0.111386,0.309428,7,0.236324,6,0.230711,...,12,0.236324,11,0.235646,14,0.239117,13,0.226278,16,0.226627
2,Jazz Musicians,louvain,4,0.261965,0.428760,0.444469,5,0.435515,6,0.434260,...,10,0.434260,11,0.434259,12,0.434258,13,0.434260,14,0.434260
3,Jazz Musicians,spectral,3,0.064143,0.115452,0.444469,4,0.294296,5,0.290681,...,9,0.288052,10,0.300420,11,0.302791,12,0.300420,13,0.296046
4,Dolphins Social Network,louvain,5,0.265447,0.508564,0.526463,6,0.509355,7,0.513686,...,11,0.523338,12,0.519304,13,0.523338,14,0.509366,15,0.523338
5,Dolphins Social Network,spectral,4,0.233397,0.353154,0.526463,5,0.478086,6,0.478086,...,10,0.478087,11,0.478086,12,0.478086,13,0.478086,14,0.478086


In [16]:
dfs.to_excel("output1.xlsx") 